<Header> This is is where we develop a baseline factor model that allows us to take certain factors and use them in multi-OLS regression to predict our excess return signals on a security level. These signals are absolute. </Header>

In [1]:
#First, import our packages for the database connection and dataframe access
import mysql.connector
import pandas as pd
import statsmodels.api as sm
import numpy as np
#pd datareader not working
#import pandas_datareader as pdr

In [2]:
# Connect to MySQL database
connection = mysql.connector.connect(
    host="ubctg.con7266gcvin.us-east-2.rds.amazonaws.com",
    user="admin",
    password="ubctgquant",
    database="ubctg"
)

Here, we pull monthly returns across the stock universe over 10 years from 2011 to 2021

In [3]:
# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Define the start and end dates
start_date = '2011-01-01'
end_date = '2021-01-31'

# SQL query to retrieve data from the "Volatility" table between two dates
sql_query = f"SELECT * FROM `Monthly Returns` mr WHERE Date BETWEEN '{start_date}' AND '{end_date}'"

# Execute the SQL query
cursor.execute(sql_query)

# Fetch all rows from the result set
universe_data = cursor.fetchall()

# Convert fetched data into a pandas DataFrame
columns = [i[0] for i in cursor.description]  # Extract column names from cursor description

#create new df
universe_df = pd.DataFrame(universe_data, columns=columns)

#close cursor and db connection
cursor.close()

# Print the DataFrame
universe_df

,PERMNO,date,COMNAM,NAICS,PERMCO,DCLRDT,DLPDT,PAYDT,RCRDDT,FACPR,FACSHR,ACPERM,ACCOMP,DLPRC,PRC,VOL,RET,BID,ASK
0,10001,2011-01-31,GAS NATURAL INC,221210,7953,2011-01-05,,2011-01-31,2011-01-14,0.0,0.0,,,NaN,10.78000,4648.0,0.028992,10.72000,10.74000
1,10001,2011-02-28,GAS NATURAL INC,221210,7953,2011-01-26,,2011-02-28,2011-02-15,0.0,0.0,,,NaN,10.98000,4586.0,0.022727,10.96000,10.99000
2,10001,2011-03-31,GAS NATURAL INC,221210,7953,2011-02-23,,2011-03-31,2011-03-15,0.0,0.0,,,NaN,11.73000,4137.0,0.072404,11.72000,11.73000
3,10001,2011-04-29,GAS NATURAL INC,221210,7953,2011-03-30,,2011-04-29,2011-04-15,0.0,0.0,,,NaN,11.23000,4109.0,-0.038789,11.21000,11.23000
4,10001,2011-05-31,GAS NATURAL INC,221210,7953,2011-04-27,,2011-05-31,2011-05-13,0.0,0.0,,,NaN,11.50000,4979.0,0.028050,11.46000,11.50000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887531,93436,2020-09-30,TESLA INC,336111,53453,,,,,NaN,NaN,,,NaN,429.01001,17331954.0,-0.139087,428.82001,429.01999
887532,93436,2020-10-30,TESLA INC,336111,53453,,,,,NaN,NaN,,,NaN,388.04001,8330610.0,-0.095499,388.04001,388.35001
887533,93436,2020-11-30,TESLA INC,336111,53453,,,,,NaN,NaN,,,NaN,567.59998,7811501.0,0.462736,566.09998,566.85999
887534,93436,2020-12-31,TESLA INC,336111,53453,,,,,NaN,NaN,,,NaN,705.66998,11962716.0,0.243252,705.21002,705.50000


Generate the number of observations per security in our table. This way we can remove those that do not have a full dataset

In [4]:
# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Define the start and end dates
start_date = '2011-01-01'
end_date = '2021-01-31'

# SQL query to retrieve data from the "Volatility" table between two dates
sql_query = f"SELECT COUNT(PERMNO), PERMNO FROM `Monthly Returns` mr WHERE Date BETWEEN '{start_date}' AND '{end_date}' GROUP BY PERMNO"

# Execute the SQL query
cursor.execute(sql_query)

# Fetch all rows from the result set
observationTable_data = cursor.fetchall()

# Convert fetched data into a pandas DataFrame
columns = [i[0] for i in cursor.description]  # Extract column names from cursor description

#create new df
observation_df = pd.DataFrame(observationTable_data, columns=columns)

#close cursor and db connection
cursor.close()

# Print the DataFrame
observation_df

unedited_observation_df = observation_df

Here, we remove securities that do not have enough observations (in our case, we look for at least 120)

In [5]:
#Filter securities with at least 120 observations in the period
observation_df_filtered = observation_df[pd.to_numeric(observation_df['COUNT(PERMNO)']) >=120]

#Inner join our dataframes to only keep the securities we have data on
universe_df_filtered = pd.merge(universe_df, observation_df_filtered, on='PERMNO', how='inner')

universe_df_filtered

,PERMNO,date,COMNAM,NAICS,PERMCO,DCLRDT,DLPDT,PAYDT,RCRDDT,FACPR,FACSHR,ACPERM,ACCOMP,DLPRC,PRC,VOL,RET,BID,ASK,COUNT(PERMNO)
0,10026,2011-01-31,J & J SNACK FOODS CORP,311821,7976,,,,,NaN,NaN,,,NaN,42.47000,12928.0,-0.119610,42.47000,42.55000,121
1,10026,2011-02-28,J & J SNACK FOODS CORP,311821,7976,,,,,NaN,NaN,,,NaN,43.97000,13651.0,0.035319,43.97000,43.99000,121
2,10026,2011-03-31,J & J SNACK FOODS CORP,311821,7976,2011-02-09,,2011-04-06,2011-03-15,0.0,0.0,,,NaN,47.07000,14227.0,0.073175,46.96000,47.08000,121
3,10026,2011-04-29,J & J SNACK FOODS CORP,311821,7976,,,,,NaN,NaN,,,NaN,50.82000,8807.0,0.079669,50.74000,50.82000,121
4,10026,2011-05-31,J & J SNACK FOODS CORP,311821,7976,,,,,NaN,NaN,,,NaN,51.52000,9658.0,0.013774,51.50000,51.52000,121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478703,93436,2020-09-30,TESLA INC,336111,53453,,,,,NaN,NaN,,,NaN,429.01001,17331954.0,-0.139087,428.82001,429.01999,121
478704,93436,2020-10-30,TESLA INC,336111,53453,,,,,NaN,NaN,,,NaN,388.04001,8330610.0,-0.095499,388.04001,388.35001,121
478705,93436,2020-11-30,TESLA INC,336111,53453,,,,,NaN,NaN,,,NaN,567.59998,7811501.0,0.462736,566.09998,566.85999,121
478706,93436,2020-12-31,TESLA INC,336111,53453,,,,,NaN,NaN,,,NaN,705.66998,11962716.0,0.243252,705.21002,705.50000,121


We can now add a few factors to our model, in this case we will use GDP, CPI (inflation data), and the unemployment rate in each month. We then add it to our monthly returns dataframe for our OLS regression

In [6]:
#Introduce our factor data below as a CSV (pandas datareader not currently working for FRED api)
macro_factors = pd.read_csv("UBCTG Factor Model Example - Macro FRED Data.csv")

#these dates are at the beginning of month, so we will operate on our monthly return dataframe to convert our dates to the beginning of the month
universe_df_filtered["date"] = pd.to_datetime(universe_df_filtered["date"]).dt.to_period('M').dt.to_timestamp()
#display(universe_df_filtered)

macro_factors.dtypes
macro_factors["date"] = pd.to_datetime(macro_factors["DATE"])
macro_factors = macro_factors.drop('DATE', axis=1)
#display(macro_factors)

universe_df_with_external_factors = pd.merge(universe_df_filtered, macro_factors, on= 'date', how='inner')

#display(universe_df_with_external_factors)

universe_df_with_external_factors[universe_df_with_external_factors.index.isin([545])]

#universe_df_with_external_factors.loc[universe_df_with_external_factors["RET"] == "I"]
universe_df_with_external_factors["RET"] = pd.to_numeric(universe_df_with_external_factors["RET"], errors="coerce")

#To do: Lag and fractionally differentiate the data

In [7]:
#columns_to_regress = ['Annualized Percent Change of GDP from Preceding Period, Seasonally Adjusted','CPI (USACPALTT01CTGYM)','UNRATE']

#Explore fractional differentiation of ask/bid or adjusted price to maintain trend in the data
new_observation_df = unedited_observation_df
uniqueTickerList = new_observation_df[pd.to_numeric(new_observation_df['COUNT(PERMNO)']) >=120]

#Unique tickers
uniqueTickerList1 = uniqueTickerList['PERMNO']
universe_df_with_external_factors.dtypes


PERMNO                                                                                  int64
date                                                                           datetime64[ns]
COMNAM                                                                                 object
NAICS                                                                                  object
PERMCO                                                                                  int64
DCLRDT                                                                                 object
DLPDT                                                                                  object
PAYDT                                                                                  object
RCRDDT                                                                                 object
FACPR                                                                                 float64
FACSHR                                                      

In [8]:
uniqueTickerList1

3        10026
4        10028
5        10032
6        10044
7        10051
         ...  
12608    93426
12609    93427
12611    93429
12616    93434
12618    93436
Name: PERMNO, Length: 3936, dtype: int64

In [9]:
universe_df_with_external_factors

,PERMNO,date,COMNAM,NAICS,PERMCO,DCLRDT,DLPDT,PAYDT,RCRDDT,FACPR,...,DLPRC,PRC,VOL,RET,BID,ASK,COUNT(PERMNO),UNRATE,CPI (USACPALTT01CTGYM),"Annualized Percent Change of GDP from Preceding Period, Seasonally Adjusted"
0,10026,2011-01-01,J & J SNACK FOODS CORP,311821,7976,,,,,NaN,...,NaN,42.47000,12928.0,-0.119610,42.47000,42.55,121,9.1,0.168134,-3.034413
1,10028,2011-01-01,D G S E COMPANIES INC,448310,7978,,,,,NaN,...,NaN,4.42000,1253.0,0.032710,4.41000,4.55,121,9.1,0.168134,-3.034413
2,10032,2011-01-01,PLEXUS CORP,334412,7980,,,,,NaN,...,NaN,27.04000,90502.0,-0.126050,27.01000,27.04,121,9.1,0.168134,-3.034413
3,10044,2011-01-01,ROCKY MOUNTAIN CHOCOLATE FAC INC,311330,7992,,,,,NaN,...,NaN,10.21280,1865.0,0.058321,10.20000,10.28,121,9.1,0.168134,-3.034413
4,10051,2011-01-01,HANGER ORTHOPEDIC GROUP INC,339113,7999,,,,,NaN,...,NaN,20.55000,32458.0,-0.030203,20.56000,20.58,121,9.1,0.168134,-3.034413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478703,93426,2021-01-01,VISHAY PRECISION GROUP INC,334416,53443,,,,,NaN,...,NaN,31.99000,14449.0,0.016201,32.00000,32.07,121,6.4,1.365916,8.270387
478704,93427,2021-01-01,FABRINET,333314,53445,,,,,NaN,...,NaN,78.94000,30462.0,0.017399,78.94000,78.95,121,6.4,1.365916,8.270387
478705,93429,2021-01-01,C B O E GLOBAL MARKETS INC,523210,53447,,,,,NaN,...,NaN,91.73000,136159.0,-0.014927,91.66000,91.73,121,6.4,1.365916,8.270387
478706,93434,2021-01-01,S & W SEED CO,111998,53427,,,,,NaN,...,NaN,3.50000,26167.0,0.194539,3.47000,3.50,121,6.4,1.365916,8.270387


In [13]:
#ticker_specific_universe_df = universe_df_with_external_factors.loc[universe_df_with_external_factors['PERMNO'] == uniqueTickerList1.iloc[0]]
#ticker_specific_universe_df

for ticker in uniqueTickerList1:
    ticker_specific_universe_df = universe_df_with_external_factors.loc[universe_df_with_external_factors['PERMNO'] == ticker]

    ticker_specific_universe_df["Alpha"] = 0
    ticker_specific_universe_df["R_squared"] = 0
    ticker_specific_universe_df["Beta_Market"] = 0
    ticker_specific_universe_df["Beta_GDP"] = 0
    ticker_specific_universe_df["Beta_UnemRate"] = 0
    ticker_specific_universe_df["p_value_Alpha"] = 0
    ticker_specific_universe_df["p_value_Market"] = 0
    ticker_specific_universe_df["p_value_GDP"] = 0
    ticker_specific_universe_df["p_value_UnemRate"] = 0
    LookBack_Window=48
    #display(ticker_specific_universe_df)
    for x in range(0, len(ticker_specific_universe_df)-LookBack_Window-1):
        # Define the independent variables (X) and dependent variable (y)
        X = ticker_specific_universe_df[["Annualized Percent Change of GDP from Preceding Period, Seasonally Adjusted","CPI (USACPALTT01CTGYM)","UNRATE"]][x:x+LookBack_Window]
        Y = ticker_specific_universe_df['RET'][x:x+LookBack_Window]

        # Add a constant term to the independent variables, check impact
        X = sm.add_constant(X)
    
        # Fit the linear regression model
        model = sm.OLS(Y, X)
        results = model.fit()
    
        # Get the coefficients
        coefficients = results.params
        
        # Get the coefficients
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'Alpha'] = results.params[0]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'Beta_GDP'] = results.params[1]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'Beta_CPI'] = results.params[2]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'Beta_UnemRate'] = results.params[3]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'R_squared'] = results.rsquared
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'p_value_Alpha'] = results.pvalues[0]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'p_value_GDP'] = results.pvalues[1]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'p_value_CPI'] = results.pvalues[2]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'p_value_UnemRate'] = results.pvalues[3]
        #display(ticker_specific_universe_df)
        #display(len(results.params))
        print(len(results.pvalues))
    break

C:\Users\clint\AppData\Local\Temp\ipykernel_15536\4219767399.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_specific_universe_df["Alpha"] = 0
C:\Users\clint\AppData\Local\Temp\ipykernel_15536\4219767399.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_specific_universe_df["R_squared"] = 0
C:\Users\clint\AppData\Local\Temp\ipykernel_15536\4219767399.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


C:\Users\clint\AppData\Local\Temp\ipykernel_15536\4219767399.py:21: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  Y = ticker_specific_universe_df['RET'][x:x+LookBack_Window]
C:\Users\clint\AppData\Local\Temp\ipykernel_15536\4219767399.py:21: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  Y = ticker_specific_universe_df['RET'][x:x+LookBack_Window]
C:\Users\clint\AppData\Local\Temp\ipykernel_15536\4219767399.py:21: FutureWarning: The behavior of `series[i:j]` with an integer-dtype i

4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


C:\Users\clint\AppData\Local\Temp\ipykernel_15536\4219767399.py:21: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  Y = ticker_specific_universe_df['RET'][x:x+LookBack_Window]
C:\Users\clint\AppData\Local\Temp\ipykernel_15536\4219767399.py:21: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  Y = ticker_specific_universe_df['RET'][x:x+LookBack_Window]
C:\Users\clint\AppData\Local\Temp\ipykernel_15536\4219767399.py:21: FutureWarning: The behavior of `series[i:j]` with an integer-dtype i

In [14]:
ticker_specific_universe_df

,PERMNO,date,COMNAM,NAICS,PERMCO,DCLRDT,DLPDT,PAYDT,RCRDDT,FACPR,...,R_squared,Beta_Market,Beta_GDP,Beta_UnemRate,p_value_Alpha,p_value_Market,p_value_GDP,p_value_UnemRate,Beta_CPI,p_value_CPI
0,10026,2011-01-01,J & J SNACK FOODS CORP,311821,7976,,,,,NaN,...,0.000000,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,NaN,NaN
3921,10026,2011-02-01,J & J SNACK FOODS CORP,311821,7976,,,,,NaN,...,0.000000,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,NaN,NaN
7864,10026,2011-03-01,J & J SNACK FOODS CORP,311821,7976,2011-02-09,,2011-04-06,2011-03-15,0.0,...,0.000000,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,NaN,NaN
11811,10026,2011-04-01,J & J SNACK FOODS CORP,311821,7976,,,,,NaN,...,0.000000,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,NaN,NaN
15747,10026,2011-05-01,J & J SNACK FOODS CORP,311821,7976,,,,,NaN,...,0.000000,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458875,10026,2020-09-01,J & J SNACK FOODS CORP,311813,7976,2020-09-17,,2020-10-13,2020-09-29,0.0,...,0.054573,0,0.000749,0.004529,0.753130,0,0.324144,0.676268,0.011462,0.829968
462814,10026,2020-10-01,J & J SNACK FOODS CORP,311813,7976,,,,,NaN,...,0.102751,0,0.000977,0.006441,0.708052,0,0.184041,0.587884,0.016377,0.789758
466748,10026,2020-11-01,J & J SNACK FOODS CORP,311813,7976,,,,,NaN,...,0.092621,0,0.000928,0.004546,0.812877,0,0.205417,0.704036,0.006912,0.912404
470689,10026,2020-12-01,J & J SNACK FOODS CORP,311813,7976,2020-12-07,,2021-01-12,2020-12-21,0.0,...,0.101061,0,0.000926,0.000288,0.871806,0,0.204746,0.981666,-0.020498,0.760959


In [12]:
for x in range(0, len(merged_data)-LookBack_Window+1):
    # Define the independent variables (X) and dependent variable (y)
    X = merged_data[['Lagged_Frac_Diff_Market','Lagged_Frac_Diff_GDP','Lagged_Frac_Diff_CPI','Lagged_Unem_Rate']][x:x+LookBack_Window]
    y = merged_data['Frac_Diff_AAPL'][x:x+LookBack_Window]



merged_data["Expected_Return"] = merged_data.filter(like="Beta").sum(axis=1)
merged_data["Tracking_Error"] = merged_data["Expected_Return"] - merged_data["Frac_Diff_AAPL"]
# Find the first non-zero and non-NaN index in the "p_value_alpha" column
first_non_zero_index = merged_data['p_value_Alpha'].ne(0).idxmax()
# Remove the rows before the first non-zero and non-NaN index
merged_data = merged_data.loc[first_non_zero_index:]
#drop the first row
merged_data = merged_data.iloc[1:]
merged_data
    
    #tickerDf = universe_df_with_external_factors("PERMNO" == ticker)
    #display(tickerDf)

NameError: name 'merged_data' is not defined